# Comparison Notebook

The last few cells of code including the big pandas dataframe compare different players

In [1]:
%matplotlib inline
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import pandas as pd

from game import Game
from player import Player

In [2]:
from your_name import YourName

In [3]:
### Generate cards from 9 to 14 (ace) for all colors/symbols (0, 1, 2, 3)
def getDeck():
    return [(number, color) for color in range(4) for number in range(9, 15)]
    
print(getDeck())

[(9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (9, 3), (10, 3), (11, 3), (12, 3), (13, 3), (14, 3)]


In [4]:
### Shuffle the cards randomly. Each player gets 9 cards
### (so one player cannot be certain which cards the other player has)

def getShuffled(deck):
    D = set(deck)
    A = set(random.sample(deck, 8))
    B = set(random.sample(list(D - A), 8))
    C = D - A - B
    if len(A.intersection(B)) > 0: print("Shuffle error 1")
    if len(A.intersection(B)) > 0: print("Shuffle error 2")
    if len(A.intersection(C)) > 0: print("Shuffle error 3") 
    DS = A | B | C
    if not DS == D: print("Shuffle error 4")  
    return list(A), list(B), list(C)

p1, p2, notUsed, = getShuffled(getDeck())
print(p1)
print(p2)


[(11, 1), (10, 1), (9, 2), (13, 0), (11, 2), (12, 2), (9, 1), (10, 2)]
[(9, 3), (13, 1), (11, 0), (12, 0), (14, 0), (13, 3), (14, 3), (13, 2)]


In [5]:
# Some examplary random player

class RandomPlayer(Player):
    
    ### player's random strategy
    def putCard(self, declared_card):
        
        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"
        
        ### player randomly decides which card put on the table
        card = random.choice(self.cards)
        declaration = card
        
        ### player randomly decides whether to cheat or not
        cheat = np.random.choice([True, False])
       
        ### if (s)he decides to cheat, (s)he randomly declares the card.
        if cheat:
            declaration = random.choice(self.cards)             
            
        ### Yet, declared card should be no worse than a card on the top of the pile . 
        if declared_card is not None and declaration[0] < declared_card[0]:
            declaration = (min(declared_card[0]+1,14), declaration[1])

        ### return the decision (true card) and declaration (player's declaration)
        return card, declaration
    
    ### randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        return np.random.choice([True, False])
    

Analyze few moves...

In [6]:
### Perform a full game 100 times
STAT_NAMES = ["Wins", "Draws", "Moves", "Cards", "Pile Size", "Checks", "Draw Decisions", "Cheats", "Errors", "Total errors"]
ANALYZE_STATS = [0, 1, 2, 3, 5, 6, 7, 8]

def printResults(results):
    print("Wins:")
    print(results[0])
    print("Draws:")
    print(results[1])
    print("Moves:")
    print(stats_moves[2])
    print("Cards:")
    print(results[3])
    print("Pile size:")
    print(results[4])
    print("Checks:")
    print(results[5])
    print("Draw decisions:")
    print(results[6])
    print("Cheats:")
    print(results[7])
    print("Errors:")
    print(results[8])
    print("Total errors:")
    print(results[9])

def comparePlayers(player1_class, player2_class):
    stats_wins = [0, 0]
    stats_draws = [0, 0]
    stats_moves = [0, 0]
    stats_cards = [0, 0]
    stats_pile_size = 0
    stats_checks = [0, 0]
    stats_draw_decisions = [0, 0]
    stats_cheats = [0, 0]
    stats_errors = [0, 0]
    
    repeats = 10000 #For most accurate tests
    errors = 0
    draws = 0

    for t in range(repeats):
        player1 = player1_class("")
        player2 = player2_class("")
        game = Game([player1, player2], log = False)
    
        error = False
        draw = False
        
        while True:
            valid, player = game.takeTurn(log = False)
            if game.moves[0] > 100 or game.moves[1] > 100:
                draws += 1
                stats_draws[0] += 1
                stats_draws[1] += 1
                if (game.player_cards[0] < game.player_cards[1]):
                    stats_wins[0] += 1
                if (game.player_cards[0] > game.player_cards[1]):
                    stats_wins[1] += 1
                    
                    
                draw=True
                #print("DRAW")
                break
            if not valid:
                error = True
                stats_errors[player] += 1
                errors += 1
                break
            if game.isFinished(log = False):
                stats_wins[player] += 1
                break
            
        stats_pile_size += len(game.pile)
        if error: continue
        #if draw: continue
       
        for j in range(2):
            stats_moves[j] += game.moves[j]
            stats_cheats[j] += game.cheats[j]
            stats_checks[j] += game.checks[j]
            stats_draw_decisions[j] += game.draw_decisions[j]
            stats_cards[j] += len(game.player_cards[j])

    div = repeats - errors
    if div > 0:
            
        stats_pile_size /= div          
        for j in range(2):
            stats_moves[j] /= div
            stats_cheats[j] /= div
            stats_checks[j] /= div
            stats_draw_decisions[j] /= div
            stats_cards[j] /= div
            
    return [stats_wins, stats_draws, stats_moves, stats_cards, stats_pile_size, stats_checks, 
            stats_draw_decisions, stats_cheats, stats_errors, errors, draws]  


# COMPARE

In [7]:
strategy = [["RandomPlayer", "RANDOM", RandomPlayer],
            ["YourName", "YourName", YourName],
           ]

In [8]:
#%pdb on
full_results = [[None for i in range(len(strategy))] for i in range(len(strategy))]

for A in range(len(strategy)):
    print("==== " + str(A), strategy[A][0])
    for B in range(A+1,len(strategy)):
        print(B, strategy[B][0])
        results = comparePlayers(strategy[A][2], strategy[B][2])
        full_results[A][B] = results
        


==== 0 RandomPlayer
1 YourName
==== 1 YourName


In [9]:
full_results

[[None,
  [[10000, 0],
   [0, 0],
   [8.0, 7.4985],
   [0.0, 15.0],
   1.0,
   [0.0, 0.0],
   [0.0, 7.4985],
   [2.6627, 0.0],
   [0, 0],
   0,
   0]],
 [None, None]]

Simple stats

In [10]:
def printMatrix(full_results, stat):
    print(STAT_NAMES[stat])
    S = " "
    for s in strategy: S += (str(s[1]) + " " )
    print(S)
    for A in range(len(strategy)):
        print(A)
        S = str(strategy[A][1]) + " "
        for B in range(len(strategy)):
            if A == B: S += "- "
            elif A < B:
                S += str(full_results[A][B][stat][0]) + " "
            else:
                S += str(full_results[B][A][stat][1]) + " "
        print(S)
    
for a in ANALYZE_STATS:
    printMatrix(full_results, a)



Wins
 RANDOM YourName 
0
RANDOM - 10000 
1
YourName 0 - 
Draws
 RANDOM YourName 
0
RANDOM - 0 
1
YourName 0 - 
Moves
 RANDOM YourName 
0
RANDOM - 8.0 
1
YourName 7.4985 - 
Cards
 RANDOM YourName 
0
RANDOM - 0.0 
1
YourName 15.0 - 
Checks
 RANDOM YourName 
0
RANDOM - 0.0 
1
YourName 0.0 - 
Draw Decisions
 RANDOM YourName 
0
RANDOM - 0.0 
1
YourName 7.4985 - 
Cheats
 RANDOM YourName 
0
RANDOM - 2.6627 
1
YourName 0.0 - 
Errors
 RANDOM YourName 
0
RANDOM - 0 
1
YourName 0 - 


### __________________________________________My Code ______________________________________________

In [11]:
agent_strategies = [["RandomPlayer", "RANDOM", RandomPlayer],
            ["YourName", "YourName", YourName],
           ["RandomPlayer2", "RANDOM", RandomPlayer]]

def compare_all(agent_strategies, verbose = 1):
    wins = {agent_strategies[i][0]: {agent_strategies[i][0]: None} for i in range(len(agent_strategies))}
    for i in range(len(agent_strategies)):
        for ii in range(i+1, len(agent_strategies)):
            player1 = agent_strategies[i]
            player2 = agent_strategies[ii]
            strategy = [player1, player2]
            for A in range(len(strategy)):
                for B in range(A+1,len(strategy)):
                    results = comparePlayers(strategy[A][2], strategy[B][2])
                    full_results[A][B] = results
                    wins[strategy[A][0]][strategy[B][0]] = full_results[0][1][0][1]
                    wins[strategy[B][0]][strategy[A][0]] = full_results[0][1][0][0]
                    if verbose:
                        if full_results[0][1][8][0] > 0 or full_results[0][1][8][1] > 0 or full_results[0][1][9] > 0:
                            print(f"Errors occured in game: {strategy[A][0]} vs {strategy[B][0]}")
    df = pd.DataFrame(wins)
    df['|| Total Wins ||'] = df.fillna(0).sum(axis=1)
    df = df.sort_values(by='|| Total Wins ||', ascending=False)

    dominated_players = []
    temp_df = df.drop(columns=["|| Total Wins ||"])
    for i, row in temp_df.iterrows():
        num_dominated = 0
        for column in temp_df.columns:
            if i != column and temp_df[column][i] > temp_df[i][column]:
                num_dominated += 1
        dominated_players.append(num_dominated)
    df['|| Dominated Players ||'] = dominated_players
    return df

compare_all(agent_strategies)

,RandomPlayer,YourName,RandomPlayer2,|| Total Wins ||,|| Dominated Players ||
RandomPlayer2,5032.0,10000.0,NaN,15032.0,2
RandomPlayer,NaN,10000.0,4968.0,14968.0,1
YourName,0.0,NaN,0.0,0.0,0


## Player Comaprison

Below I have tested the players against each other, and different simple heuristics as well as my solutions. The dataframe orders players according to the total number of wins against remaining players ( the "|| Total Wins ||" column), top players are the best. However, you can also check how many players the agent was better than - "|| Dominated Players ||". The numbers in remaining columns show how many times your agent has won against someone, so if you look at the row from dataframe above as an example the agent called "RandomPlayer" and compare it with column "RandomPlayer2" you can see that RandomPlayer won against the RandomPlayer2 4968 games out of 10000, you can also see that RandomPlayer2 won 5032 games against RandomPlayer, etc.

Below the dataframe I also print for some most important players their row values to see which agents they won against and which they won against, etc.

In [12]:
from Test_Players import *
from B import *
from adamK import *
from Adam_player import *
from ensamble import *
from Michal_Janek_ver_1 import *
from Michal_Janek_ver_2 import *
from Vadym_Player import *
from Krzysztof_Player import *

In [13]:
agent_strategies = [["RandomPlayer", "RANDOM", RandomPlayer],
            ["Saint_Sceptic_25_Naive", "SAINT", Saint_Sceptic_25_Naive],
            ["Saint_Sceptic_50_Naive", "SAINT", Saint_Sceptic_50_Naive],
            ["Saint_Sceptic_75_Naive", "SAINT", Saint_Sceptic_75_Naive],
            ["Saint_Sceptic_Adaptable_Naive", "SAINT", Saint_Sceptic_Adaptable_Naive],
            ["Mat_Player_ver_4", "ENSAMBLE", ensamble_player],
            ["YourName - not implemented Player", "YourName", YourName],
           ["MyRandom_Default", "CUSTOM_RANDOM", MyRandom_Default],
           ["MyRandom_All_Cards", "CUSTOM_RANDOM", MyRandom_All_Cards],
           ["MyRandom_Real_Cards", "CUSTOM_RANDOM", MyRandom_Real_Cards],
           ["Random_Never_Accuse", "CUSTOM_RANDOM", Random_Never_Accuse],
           ["Random_Always_Accuse", "CUSTOM_RANDOM", Random_Always_Accuse],
           ["Mimic", "CUSTOM_RANDOM", Mimic],
           ["Saint_Believer", "SAINT", Saint_Believer],
           ["Saint_Once_in_a_While", "SAINT", Saint_Once_in_a_While],
           ["Saint_Two_Face", "SAINT", Saint_Two_Face],
           ["Saint_Accountant", "SAINT", Saint_Accountant],
           ["Saint_Nervous", "SAINT", Saint_Nervous],
           ["Mat_Player_ver_1", "SAINT", Saint_Nervous_ver_2],
           ["Mat_Player_ver_2", "SAINT", Saint_Nervous_ver_3],
           ["Mat_Player_ver_3", "SAINT", Saint_Nervous_Memory],
           ["Adam_M_Player", "CUSTOM", Adam_player],
           ["AdamK_ver_1", "CUSTOM", AdamK],
           ["AdamK2_ver_2", "CUSTOM", AdamK2],
           ["Saint_Collector", "SAINT", Saint_Collector],
           ["Dabbler_Believer", "DABBLER", Dabbler_Believer],
           ["Dabbler_Once_in_a_While", "DABBLER", Dabbler_Once_in_a_While],
           ["Dabbler_Two_Face", "DABBLER", Dabbler_Two_Face],
           ["Dabbler_Accountant", "DABBLER", Dabbler_Accountant],
           ["Dabbler_Nervous", "DABBLER", Dabbler_Nervous],
           ["Dabbler_Collector", "DABBLER", Dabbler_Collector],
           ["Coin_Flipper_Believer", "COIN_FLIPPER", Coin_Flipper_Believer],
           ["Coin_Flipper_Once_in_a_While", "COIN_FLIPPER", Coin_Flipper_Once_in_a_While],
           ["Coin_Flipper_Two_Face", "COIN_FLIPPER", Coin_Flipper_Two_Face],
           ["Coin_Flipper_Accountant", "COIN_FLIPPER", Coin_Flipper_Accountant],
           ["Coin_Flipper_Nervous", "COIN_FLIPPER", Coin_Flipper_Nervous],
           ["Coin_Flipper_Collector", "COIN_FLIPPER", Coin_Flipper_Collector],
           ["Pathological_Believer", "PATHOLOGICAL", Pathological_Believer],
           ["Pathological_Once_in_a_While", "PATHOLOGICAL", Pathological_Once_in_a_While],
           ["Pathological_Two_Face", "PATHOLOGICAL", Pathological_Two_Face],
           ["Pathological_Accountant", "PATHOLOGICAL", Pathological_Accountant],
           ["Pathological_Nervous", "PUSHER", Pathological_Nervous],
           ["Pathological_Collector", "PUSHER", Pathological_Collector],
           ["Pusher_Believer", "PUSHER", Pusher_Believer],
           ["Pusher_Once_in_a_While", "PUSHER", Pusher_Once_in_a_While],
           ["Pusher_Two_Face", "PUSHER", Pusher_Two_Face],
           ["Pusher_Accountant", "PUSHER", Pusher_Accountant],
           ["Pusher_Nervous", "PUSHER", Pusher_Nervous],
           ["Pusher_Collector", "PUSHER", Pusher_Collector],
           ["B_1", "B_PLAYER", Blayer],
           ["B_2", "B_PLAYER", Blayer2],
           ["B_3", "B_PLAYER", Blayer3],
           ["Bartek_ver_1", "B_PLAYER", Saint_Nervous_ver_B],
           ["Bartek_ver_2", "B_PLAYER", Saint_Nervous_ver_B2],
           ["Bartek_ver_3", "B_PLAYER", Saint_Nervous_ver_B3],
           ["Bartek_ver_4", "B_PLAYER", Saint_Nervous_ver_B4],
           ["Janek_Michal_ver_1", "MICHAL_JANEK", KaminskiIndrzejczak_ver_1],
           ["Janek_Michal_ver_2", "MICHAL_JANEK", KaminskiIndrzejczak_ver_2],
           ["Vadym_Player", "VADYM", Vadym_Player],
           ["Krzysztof_Player", "KRZYSZTOF", Krzysztof_Player]
           ]

results = compare_all(agent_strategies)
results

,RandomPlayer,Saint_Sceptic_25_Naive,Saint_Sceptic_50_Naive,Saint_Sceptic_75_Naive,Saint_Sceptic_Adaptable_Naive,Mat_Player_ver_4,YourName - not implemented Player,MyRandom_Default,MyRandom_All_Cards,MyRandom_Real_Cards,...,Bartek_ver_1,Bartek_ver_2,Bartek_ver_3,Bartek_ver_4,Janek_Michal_ver_1,Janek_Michal_ver_2,Vadym_Player,Krzysztof_Player,|| Total Wins ||,|| Dominated Players ||
Mat_Player_ver_4,9999.0,10000.0,10000.0,10000.0,8289.0,NaN,10000.0,10000.0,10000.0,9999.0,...,5007.0,4930.0,5349.0,5490.0,10000.0,3974.0,9178.0,10000.0,506866.0,56
Bartek_ver_2,10000.0,10000.0,10000.0,10000.0,8335.0,5070.0,10000.0,10000.0,10000.0,10000.0,...,5053.0,NaN,5376.0,5340.0,10000.0,3971.0,9185.0,10000.0,506851.0,58
Bartek_ver_1,9999.0,10000.0,10000.0,10000.0,8246.0,4993.0,10000.0,10000.0,10000.0,9999.0,...,NaN,4947.0,5283.0,5493.0,10000.0,3997.0,9242.0,10000.0,506558.0,53
Mat_Player_ver_2,10000.0,10000.0,10000.0,10000.0,8223.0,4998.0,10000.0,10000.0,10000.0,9999.0,...,5022.0,4988.0,4484.0,4445.0,10000.0,3975.0,9156.0,10000.0,501512.0,53
Mat_Player_ver_1,9998.0,10000.0,10000.0,10000.0,8201.0,5117.0,10000.0,10000.0,10000.0,9997.0,...,5048.0,4930.0,4427.0,4393.0,10000.0,3996.0,9148.0,10000.0,501159.0,52
Saint_Nervous,9995.0,10000.0,10000.0,10000.0,8193.0,4990.0,10000.0,9999.0,10000.0,9998.0,...,5072.0,4982.0,3659.0,3758.0,10000.0,3843.0,9070.0,10000.0,489477.0,52
Bartek_ver_3,9283.0,9267.0,9264.0,9280.0,9692.0,4651.0,10000.0,9269.0,9937.0,7974.0,...,4717.0,4624.0,NaN,5079.0,9652.0,3946.0,3838.0,9269.0,475150.0,53
Bartek_ver_4,9212.0,9299.0,9253.0,9270.0,9741.0,4510.0,10000.0,9155.0,9939.0,7412.0,...,4507.0,4660.0,4921.0,NaN,9504.0,4034.0,3856.0,9252.0,473168.0,52
Mat_Player_ver_3,9998.0,9185.0,9201.0,9209.0,5935.0,2474.0,10000.0,10000.0,10000.0,9999.0,...,2500.0,2520.0,4587.0,4495.0,7437.0,3976.0,9059.0,9194.0,457135.0,45
B_3,8923.0,9266.0,9254.0,9286.0,9120.0,4436.0,10000.0,8975.0,9883.0,7335.0,...,4459.0,4594.0,5034.0,5012.0,9606.0,3825.0,3408.0,9245.0,446002.0,53


### Player-Wise Comparisons

In [40]:
for col, val in results.loc["Janek_Michal_ver_1"].items():
    print(f'{col}: {val}')

RandomPlayer: 9807.0
Saint_Sceptic_25_Naive: 5709.0
Saint_Sceptic_50_Naive: 5633.0
Saint_Sceptic_75_Naive: 5524.0
Saint_Sceptic_Adaptable_Naive: 0.0
Mat_Player_ver_4: 0.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 9993.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 9751.0
Random_Never_Accuse: 9882.0
Random_Always_Accuse: 9998.0
Mimic: 10000.0
Saint_Believer: 522.0
Saint_Once_in_a_While: 22.0
Saint_Two_Face: 0.0
Saint_Accountant: 0.0
Saint_Nervous: 0.0
Mat_Player_ver_1: 0.0
Mat_Player_ver_2: 0.0
Mat_Player_ver_3: 2563.0
Adam_M_Player: 5620.0
AdamK_ver_1: 274.0
AdamK2_ver_2: 2312.0
Saint_Collector: 568.0
Dabbler_Believer: 3106.0
Dabbler_Once_in_a_While: 3641.0
Dabbler_Two_Face: 6330.0
Dabbler_Accountant: 5328.0
Dabbler_Nervous: 2140.0
Dabbler_Collector: 3315.0
Coin_Flipper_Believer: 8575.0
Coin_Flipper_Once_in_a_While: 8915.0
Coin_Flipper_Two_Face: 9794.0
Coin_Flipper_Accountant: 9407.0
Coin_Flipper_Nervous: 8190.0
Coin_Flipper_Collector: 8645.0
Pathological_Believe

In [41]:
for col, val in results.loc["Janek_Michal_ver_2"].items():
    print(f'{col}: {val}')

RandomPlayer: 6495.0
Saint_Sceptic_25_Naive: 9191.0
Saint_Sceptic_50_Naive: 9283.0
Saint_Sceptic_75_Naive: 9375.0
Saint_Sceptic_Adaptable_Naive: 8475.0
Mat_Player_ver_4: 6026.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 8076.0
MyRandom_All_Cards: 9690.0
MyRandom_Real_Cards: 4197.0
Random_Never_Accuse: 8731.0
Random_Always_Accuse: 5568.0
Mimic: 9944.0
Saint_Believer: 7536.0
Saint_Once_in_a_While: 7474.0
Saint_Two_Face: 7974.0
Saint_Accountant: 7843.0
Saint_Nervous: 6157.0
Mat_Player_ver_1: 6004.0
Mat_Player_ver_2: 6025.0
Mat_Player_ver_3: 6024.0
Adam_M_Player: 8046.0
AdamK_ver_1: 5940.0
AdamK2_ver_2: 6662.0
Saint_Collector: 7406.0
Dabbler_Believer: 8869.0
Dabbler_Once_in_a_While: 8553.0
Dabbler_Two_Face: 7100.0
Dabbler_Accountant: 7004.0
Dabbler_Nervous: 6192.0
Dabbler_Collector: 8932.0
Coin_Flipper_Believer: 8796.0
Coin_Flipper_Once_in_a_While: 8451.0
Coin_Flipper_Two_Face: 6472.0
Coin_Flipper_Accountant: 6306.0
Coin_Flipper_Nervous: 6211.0
Coin_Flipper_Collector: 875

In [42]:
for col, val in results.loc["Vadym_Player"].items():
    print(f'{col}: {val}')

RandomPlayer: 6185.0
Saint_Sceptic_25_Naive: 4346.0
Saint_Sceptic_50_Naive: 4990.0
Saint_Sceptic_75_Naive: 5775.0
Saint_Sceptic_Adaptable_Naive: 7103.0
Mat_Player_ver_4: 822.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 8272.0
MyRandom_All_Cards: 9865.0
MyRandom_Real_Cards: 4749.0
Random_Never_Accuse: 9445.0
Random_Always_Accuse: 878.0
Mimic: 9993.0
Saint_Believer: 7341.0
Saint_Once_in_a_While: 6558.0
Saint_Two_Face: 4524.0
Saint_Accountant: 4954.0
Saint_Nervous: 930.0
Mat_Player_ver_1: 852.0
Mat_Player_ver_2: 844.0
Mat_Player_ver_3: 941.0
Adam_M_Player: 3262.0
AdamK_ver_1: 5683.0
AdamK2_ver_2: 6432.0
Saint_Collector: 7021.0
Dabbler_Believer: 9314.0
Dabbler_Once_in_a_While: 8983.0
Dabbler_Two_Face: 6141.0
Dabbler_Accountant: 4807.0
Dabbler_Nervous: 4728.0
Dabbler_Collector: 9225.0
Coin_Flipper_Believer: 9370.0
Coin_Flipper_Once_in_a_While: 9170.0
Coin_Flipper_Two_Face: 6209.0
Coin_Flipper_Accountant: 4613.0
Coin_Flipper_Nervous: 5292.0
Coin_Flipper_Collector: 9268.0
Pa

In [43]:
for col, val in results.loc["Krzysztof_Player"].items():
    print(f'{col}: {val}')

RandomPlayer: 9924.0
Saint_Sceptic_25_Naive: 5077.0
Saint_Sceptic_50_Naive: 4934.0
Saint_Sceptic_75_Naive: 4905.0
Saint_Sceptic_Adaptable_Naive: 0.0
Mat_Player_ver_4: 0.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 10000.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 10000.0
Random_Never_Accuse: 9999.0
Random_Always_Accuse: 10000.0
Mimic: 10000.0
Saint_Believer: 0.0
Saint_Once_in_a_While: 0.0
Saint_Two_Face: 0.0
Saint_Accountant: 0.0
Saint_Nervous: 0.0
Mat_Player_ver_1: 0.0
Mat_Player_ver_2: 0.0
Mat_Player_ver_3: 806.0
Adam_M_Player: 5650.0
AdamK_ver_1: 286.0
AdamK2_ver_2: 297.0
Saint_Collector: 129.0
Dabbler_Believer: 2754.0
Dabbler_Once_in_a_While: 2945.0
Dabbler_Two_Face: 3729.0
Dabbler_Accountant: 4635.0
Dabbler_Nervous: 1483.0
Dabbler_Collector: 2683.0
Coin_Flipper_Believer: 8729.0
Coin_Flipper_Once_in_a_While: 9101.0
Coin_Flipper_Two_Face: 9901.0
Coin_Flipper_Accountant: 9585.0
Coin_Flipper_Nervous: 8630.0
Coin_Flipper_Collector: 8956.0
Pathological_Believer:

In [44]:
for col, val in results.loc["Adam_M_Player"].items():
    print(f'{col}: {val}')

RandomPlayer: 9982.0
Saint_Sceptic_25_Naive: 4302.0
Saint_Sceptic_50_Naive: 4307.0
Saint_Sceptic_75_Naive: 4289.0
Saint_Sceptic_Adaptable_Naive: 81.0
Mat_Player_ver_4: 16.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 10000.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 9999.0
Random_Never_Accuse: 9974.0
Random_Always_Accuse: 10000.0
Mimic: 10000.0
Saint_Believer: 66.0
Saint_Once_in_a_While: 54.0
Saint_Two_Face: 9.0
Saint_Accountant: 46.0
Saint_Nervous: 7.0
Mat_Player_ver_1: 9.0
Mat_Player_ver_2: 13.0
Mat_Player_ver_3: 814.0
Adam_M_Player: nan
AdamK_ver_1: 602.0
AdamK2_ver_2: 598.0
Saint_Collector: 73.0
Dabbler_Believer: 5380.0
Dabbler_Once_in_a_While: 6547.0
Dabbler_Two_Face: 9413.0
Dabbler_Accountant: 7687.0
Dabbler_Nervous: 4013.0
Dabbler_Collector: 5607.0
Coin_Flipper_Believer: 9187.0
Coin_Flipper_Once_in_a_While: 9551.0
Coin_Flipper_Two_Face: 9985.0
Coin_Flipper_Accountant: 9873.0
Coin_Flipper_Nervous: 9007.0
Coin_Flipper_Collector: 9199.0
Pathological_Believer

In [45]:
for col, va in results.loc["AdamK_ver_1"].items():
    print(f'{col}: {val}')

RandomPlayer: 32.0
Saint_Sceptic_25_Naive: 32.0
Saint_Sceptic_50_Naive: 32.0
Saint_Sceptic_75_Naive: 32.0
Saint_Sceptic_Adaptable_Naive: 32.0
Mat_Player_ver_4: 32.0
YourName - not implemented Player: 32.0
MyRandom_Default: 32.0
MyRandom_All_Cards: 32.0
MyRandom_Real_Cards: 32.0
Random_Never_Accuse: 32.0
Random_Always_Accuse: 32.0
Mimic: 32.0
Saint_Believer: 32.0
Saint_Once_in_a_While: 32.0
Saint_Two_Face: 32.0
Saint_Accountant: 32.0
Saint_Nervous: 32.0
Mat_Player_ver_1: 32.0
Mat_Player_ver_2: 32.0
Mat_Player_ver_3: 32.0
Adam_M_Player: 32.0
AdamK_ver_1: 32.0
AdamK2_ver_2: 32.0
Saint_Collector: 32.0
Dabbler_Believer: 32.0
Dabbler_Once_in_a_While: 32.0
Dabbler_Two_Face: 32.0
Dabbler_Accountant: 32.0
Dabbler_Nervous: 32.0
Dabbler_Collector: 32.0
Coin_Flipper_Believer: 32.0
Coin_Flipper_Once_in_a_While: 32.0
Coin_Flipper_Two_Face: 32.0
Coin_Flipper_Accountant: 32.0
Coin_Flipper_Nervous: 32.0
Coin_Flipper_Collector: 32.0
Pathological_Believer: 32.0
Pathological_Once_in_a_While: 32.0
Patholog

In [46]:
for col, val in results.loc["AdamK2_ver_2"].items():
    print(f'{col}: {val}')

RandomPlayer: 9269.0
Saint_Sceptic_25_Naive: 9554.0
Saint_Sceptic_50_Naive: 9523.0
Saint_Sceptic_75_Naive: 9622.0
Saint_Sceptic_Adaptable_Naive: 7760.0
Mat_Player_ver_4: 3202.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 9598.0
MyRandom_All_Cards: 9963.0
MyRandom_Real_Cards: 8368.0
Random_Never_Accuse: 8649.0
Random_Always_Accuse: 9188.0
Mimic: 9986.0
Saint_Believer: 6032.0
Saint_Once_in_a_While: 6843.0
Saint_Two_Face: 8441.0
Saint_Accountant: 8458.0
Saint_Nervous: 3664.0
Mat_Player_ver_1: 3701.0
Mat_Player_ver_2: 3705.0
Mat_Player_ver_3: 5363.0
Adam_M_Player: 9402.0
AdamK_ver_1: 4503.0
AdamK2_ver_2: nan
Saint_Collector: 6144.0
Dabbler_Believer: 8160.0
Dabbler_Once_in_a_While: 8266.0
Dabbler_Two_Face: 9380.0
Dabbler_Accountant: 8988.0
Dabbler_Nervous: 6752.0
Dabbler_Collector: 8214.0
Coin_Flipper_Believer: 8470.0
Coin_Flipper_Once_in_a_While: 8415.0
Coin_Flipper_Two_Face: 9258.0
Coin_Flipper_Accountant: 9041.0
Coin_Flipper_Nervous: 7425.0
Coin_Flipper_Collector: 8438.0

In [47]:
for col, val in results.loc["B_1"].items():
    print(f'{col}: {val}')

RandomPlayer: 8557.0
Saint_Sceptic_25_Naive: 10000.0
Saint_Sceptic_50_Naive: 10000.0
Saint_Sceptic_75_Naive: 10000.0
Saint_Sceptic_Adaptable_Naive: 6532.0
Mat_Player_ver_4: 3461.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 9526.0
MyRandom_All_Cards: 9962.0
MyRandom_Real_Cards: 6346.0
Random_Never_Accuse: 3528.0
Random_Always_Accuse: 9913.0
Mimic: 10000.0
Saint_Believer: 4184.0
Saint_Once_in_a_While: 5672.0
Saint_Two_Face: 8832.0
Saint_Accountant: 8176.0
Saint_Nervous: 3401.0
Mat_Player_ver_1: 3571.0
Mat_Player_ver_2: 3521.0
Mat_Player_ver_3: 6438.0
Adam_M_Player: 9961.0
AdamK_ver_1: 4125.0
AdamK2_ver_2: 5177.0
Saint_Collector: 4238.0
Dabbler_Believer: 4113.0
Dabbler_Once_in_a_While: 4935.0
Dabbler_Two_Face: 9130.0
Dabbler_Accountant: 5672.0
Dabbler_Nervous: 3057.0
Dabbler_Collector: 4228.0
Coin_Flipper_Believer: 3822.0
Coin_Flipper_Once_in_a_While: 4567.0
Coin_Flipper_Two_Face: 8613.0
Coin_Flipper_Accountant: 5111.0
Coin_Flipper_Nervous: 3065.0
Coin_Flipper_Collector:

In [48]:
for col, val in results.loc["B_2"].items():
    print(f'{col}: {val}')

RandomPlayer: 8255.0
Saint_Sceptic_25_Naive: 10000.0
Saint_Sceptic_50_Naive: 10000.0
Saint_Sceptic_75_Naive: 9999.0
Saint_Sceptic_Adaptable_Naive: 6453.0
Mat_Player_ver_4: 3468.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 9368.0
MyRandom_All_Cards: 9967.0
MyRandom_Real_Cards: 5100.0
Random_Never_Accuse: 3405.0
Random_Always_Accuse: 9913.0
Mimic: 10000.0
Saint_Believer: 4079.0
Saint_Once_in_a_While: 5635.0
Saint_Two_Face: 8827.0
Saint_Accountant: 8018.0
Saint_Nervous: 3426.0
Mat_Player_ver_1: 3515.0
Mat_Player_ver_2: 3482.0
Mat_Player_ver_3: 6457.0
Adam_M_Player: 9946.0
AdamK_ver_1: 4097.0
AdamK2_ver_2: 5153.0
Saint_Collector: 4296.0
Dabbler_Believer: 3972.0
Dabbler_Once_in_a_While: 4815.0
Dabbler_Two_Face: 8907.0
Dabbler_Accountant: 5548.0
Dabbler_Nervous: 2966.0
Dabbler_Collector: 4098.0
Coin_Flipper_Believer: 3678.0
Coin_Flipper_Once_in_a_While: 4342.0
Coin_Flipper_Two_Face: 8206.0
Coin_Flipper_Accountant: 5026.0
Coin_Flipper_Nervous: 2932.0
Coin_Flipper_Collector: 

In [49]:
for col, val in results.loc["B_3"].items():
    print(f'{col}: {val}')

RandomPlayer: 8923.0
Saint_Sceptic_25_Naive: 9266.0
Saint_Sceptic_50_Naive: 9254.0
Saint_Sceptic_75_Naive: 9286.0
Saint_Sceptic_Adaptable_Naive: 9120.0
Mat_Player_ver_4: 4436.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 8975.0
MyRandom_All_Cards: 9883.0
MyRandom_Real_Cards: 7335.0
Random_Never_Accuse: 8723.0
Random_Always_Accuse: 7792.0
Mimic: 9864.0
Saint_Believer: 7614.0
Saint_Once_in_a_While: 7892.0
Saint_Two_Face: 9272.0
Saint_Accountant: 9457.0
Saint_Nervous: 6242.0
Mat_Player_ver_1: 5470.0
Mat_Player_ver_2: 5457.0
Mat_Player_ver_3: 5337.0
Adam_M_Player: 9291.0
AdamK_ver_1: 5185.0
AdamK2_ver_2: 5637.0
Saint_Collector: 7430.0
Dabbler_Believer: 8903.0
Dabbler_Once_in_a_While: 8738.0
Dabbler_Two_Face: 9205.0
Dabbler_Accountant: 9447.0
Dabbler_Nervous: 7218.0
Dabbler_Collector: 8867.0
Coin_Flipper_Believer: 8836.0
Coin_Flipper_Once_in_a_While: 8606.0
Coin_Flipper_Two_Face: 8955.0
Coin_Flipper_Accountant: 9289.0
Coin_Flipper_Nervous: 7210.0
Coin_Flipper_Collector: 879

In [50]:
for col, val in results.loc["Bartek_ver_1"].items():
    print(f'{col}: {val}')

RandomPlayer: 9999.0
Saint_Sceptic_25_Naive: 10000.0
Saint_Sceptic_50_Naive: 10000.0
Saint_Sceptic_75_Naive: 10000.0
Saint_Sceptic_Adaptable_Naive: 8246.0
Mat_Player_ver_4: 4993.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 10000.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 9999.0
Random_Never_Accuse: 9909.0
Random_Always_Accuse: 10000.0
Mimic: 10000.0
Saint_Believer: 5933.0
Saint_Once_in_a_While: 7758.0
Saint_Two_Face: 9966.0
Saint_Accountant: 9421.0
Saint_Nervous: 4928.0
Mat_Player_ver_1: 4952.0
Mat_Player_ver_2: 4978.0
Mat_Player_ver_3: 7500.0
Adam_M_Player: 9987.0
AdamK_ver_1: 6412.0
AdamK2_ver_2: 6797.0
Saint_Collector: 6010.0
Dabbler_Believer: 8669.0
Dabbler_Once_in_a_While: 9445.0
Dabbler_Two_Face: 9999.0
Dabbler_Accountant: 9851.0
Dabbler_Nervous: 8416.0
Dabbler_Collector: 8672.0
Coin_Flipper_Believer: 9491.0
Coin_Flipper_Once_in_a_While: 9813.0
Coin_Flipper_Two_Face: 9998.0
Coin_Flipper_Accountant: 9966.0
Coin_Flipper_Nervous: 9490.0
Coin_Flipper_Collect

In [51]:
for col, val in results.loc["Bartek_ver_2"].items():
    print(f'{col}: {val}')

RandomPlayer: 10000.0
Saint_Sceptic_25_Naive: 10000.0
Saint_Sceptic_50_Naive: 10000.0
Saint_Sceptic_75_Naive: 10000.0
Saint_Sceptic_Adaptable_Naive: 8335.0
Mat_Player_ver_4: 5070.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 10000.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 10000.0
Random_Never_Accuse: 9917.0
Random_Always_Accuse: 10000.0
Mimic: 10000.0
Saint_Believer: 6004.0
Saint_Once_in_a_While: 7724.0
Saint_Two_Face: 9958.0
Saint_Accountant: 9413.0
Saint_Nervous: 5018.0
Mat_Player_ver_1: 5070.0
Mat_Player_ver_2: 5012.0
Mat_Player_ver_3: 7480.0
Adam_M_Player: 9990.0
AdamK_ver_1: 6509.0
AdamK2_ver_2: 6757.0
Saint_Collector: 5959.0
Dabbler_Believer: 8740.0
Dabbler_Once_in_a_While: 9496.0
Dabbler_Two_Face: 9997.0
Dabbler_Accountant: 9876.0
Dabbler_Nervous: 8378.0
Dabbler_Collector: 8721.0
Coin_Flipper_Believer: 9503.0
Coin_Flipper_Once_in_a_While: 9806.0
Coin_Flipper_Two_Face: 9997.0
Coin_Flipper_Accountant: 9968.0
Coin_Flipper_Nervous: 9470.0
Coin_Flipper_Colle

In [52]:
for col, val in results.loc["Bartek_ver_3"].items():
    print(f'{col}: {val}')

RandomPlayer: 9283.0
Saint_Sceptic_25_Naive: 9267.0
Saint_Sceptic_50_Naive: 9264.0
Saint_Sceptic_75_Naive: 9280.0
Saint_Sceptic_Adaptable_Naive: 9692.0
Mat_Player_ver_4: 4651.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 9269.0
MyRandom_All_Cards: 9937.0
MyRandom_Real_Cards: 7974.0
Random_Never_Accuse: 9736.0
Random_Always_Accuse: 7933.0
Mimic: 9923.0
Saint_Believer: 7858.0
Saint_Once_in_a_While: 8117.0
Saint_Two_Face: 9371.0
Saint_Accountant: 9648.0
Saint_Nervous: 6341.0
Mat_Player_ver_1: 5573.0
Mat_Player_ver_2: 5516.0
Mat_Player_ver_3: 5413.0
Adam_M_Player: 9428.0
AdamK_ver_1: 5425.0
AdamK2_ver_2: 5783.0
Saint_Collector: 7759.0
Dabbler_Believer: 9755.0
Dabbler_Once_in_a_While: 9497.0
Dabbler_Two_Face: 9518.0
Dabbler_Accountant: 9800.0
Dabbler_Nervous: 7905.0
Dabbler_Collector: 9742.0
Coin_Flipper_Believer: 9739.0
Coin_Flipper_Once_in_a_While: 9468.0
Coin_Flipper_Two_Face: 9379.0
Coin_Flipper_Accountant: 9735.0
Coin_Flipper_Nervous: 8017.0
Coin_Flipper_Collector: 969

In [53]:
for col, val in results.loc["Bartek_ver_4"].items():
    print(f'{col}: {val}')

RandomPlayer: 9212.0
Saint_Sceptic_25_Naive: 9299.0
Saint_Sceptic_50_Naive: 9253.0
Saint_Sceptic_75_Naive: 9270.0
Saint_Sceptic_Adaptable_Naive: 9741.0
Mat_Player_ver_4: 4510.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 9155.0
MyRandom_All_Cards: 9939.0
MyRandom_Real_Cards: 7412.0
Random_Never_Accuse: 9703.0
Random_Always_Accuse: 8017.0
Mimic: 9919.0
Saint_Believer: 7905.0
Saint_Once_in_a_While: 8214.0
Saint_Two_Face: 9361.0
Saint_Accountant: 9653.0
Saint_Nervous: 6242.0
Mat_Player_ver_1: 5607.0
Mat_Player_ver_2: 5555.0
Mat_Player_ver_3: 5505.0
Adam_M_Player: 9352.0
AdamK_ver_1: 5365.0
AdamK2_ver_2: 5806.0
Saint_Collector: 7789.0
Dabbler_Believer: 9766.0
Dabbler_Once_in_a_While: 9527.0
Dabbler_Two_Face: 9454.0
Dabbler_Accountant: 9811.0
Dabbler_Nervous: 7877.0
Dabbler_Collector: 9761.0
Coin_Flipper_Believer: 9759.0
Coin_Flipper_Once_in_a_While: 9479.0
Coin_Flipper_Two_Face: 9241.0
Coin_Flipper_Accountant: 9714.0
Coin_Flipper_Nervous: 8069.0
Coin_Flipper_Collector: 967

In [54]:
for col, val in results.loc["Mat_Player_ver_1"].items():
    print(f'{col}: {val}')

RandomPlayer: 9998.0
Saint_Sceptic_25_Naive: 10000.0
Saint_Sceptic_50_Naive: 10000.0
Saint_Sceptic_75_Naive: 10000.0
Saint_Sceptic_Adaptable_Naive: 8201.0
Mat_Player_ver_4: 5117.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 10000.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 9997.0
Random_Never_Accuse: 9909.0
Random_Always_Accuse: 10000.0
Mimic: 9636.0
Saint_Believer: 5913.0
Saint_Once_in_a_While: 7649.0
Saint_Two_Face: 9964.0
Saint_Accountant: 9422.0
Saint_Nervous: 4977.0
Mat_Player_ver_1: nan
Mat_Player_ver_2: 4979.0
Mat_Player_ver_3: 7495.0
Adam_M_Player: 9991.0
AdamK_ver_1: 6450.0
AdamK2_ver_2: 6299.0
Saint_Collector: 5981.0
Dabbler_Believer: 8450.0
Dabbler_Once_in_a_While: 9352.0
Dabbler_Two_Face: 9997.0
Dabbler_Accountant: 9831.0
Dabbler_Nervous: 8175.0
Dabbler_Collector: 8498.0
Coin_Flipper_Believer: 9410.0
Coin_Flipper_Once_in_a_While: 9764.0
Coin_Flipper_Two_Face: 9999.0
Coin_Flipper_Accountant: 9964.0
Coin_Flipper_Nervous: 9434.0
Coin_Flipper_Collector: 

In [55]:
for col, val in results.loc["Mat_Player_ver_2"].items():
    print(f'{col}: {val}')

RandomPlayer: 10000.0
Saint_Sceptic_25_Naive: 10000.0
Saint_Sceptic_50_Naive: 10000.0
Saint_Sceptic_75_Naive: 10000.0
Saint_Sceptic_Adaptable_Naive: 8223.0
Mat_Player_ver_4: 4998.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 10000.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 9999.0
Random_Never_Accuse: 9903.0
Random_Always_Accuse: 10000.0
Mimic: 9642.0
Saint_Believer: 5932.0
Saint_Once_in_a_While: 7720.0
Saint_Two_Face: 9960.0
Saint_Accountant: 9427.0
Saint_Nervous: 5001.0
Mat_Player_ver_1: 5021.0
Mat_Player_ver_2: nan
Mat_Player_ver_3: 7498.0
Adam_M_Player: 9987.0
AdamK_ver_1: 6390.0
AdamK2_ver_2: 6295.0
Saint_Collector: 5986.0
Dabbler_Believer: 8490.0
Dabbler_Once_in_a_While: 9407.0
Dabbler_Two_Face: 9994.0
Dabbler_Accountant: 9835.0
Dabbler_Nervous: 8227.0
Dabbler_Collector: 8469.0
Coin_Flipper_Believer: 9427.0
Coin_Flipper_Once_in_a_While: 9747.0
Coin_Flipper_Two_Face: 9997.0
Coin_Flipper_Accountant: 9965.0
Coin_Flipper_Nervous: 9386.0
Coin_Flipper_Collector:

In [56]:
for col, val in results.loc["Mat_Player_ver_3"].items():
    print(f'{col}: {val}')

RandomPlayer: 9998.0
Saint_Sceptic_25_Naive: 9185.0
Saint_Sceptic_50_Naive: 9201.0
Saint_Sceptic_75_Naive: 9209.0
Saint_Sceptic_Adaptable_Naive: 5935.0
Mat_Player_ver_4: 2474.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 10000.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 9999.0
Random_Never_Accuse: 9912.0
Random_Always_Accuse: 10000.0
Mimic: 10000.0
Saint_Believer: 5043.0
Saint_Once_in_a_While: 5613.0
Saint_Two_Face: 9260.0
Saint_Accountant: 6463.0
Saint_Nervous: 2613.0
Mat_Player_ver_1: 2505.0
Mat_Player_ver_2: 2502.0
Mat_Player_ver_3: nan
Adam_M_Player: 9186.0
AdamK_ver_1: 4674.0
AdamK2_ver_2: 4637.0
Saint_Collector: 5187.0
Dabbler_Believer: 8034.0
Dabbler_Once_in_a_While: 9132.0
Dabbler_Two_Face: 9996.0
Dabbler_Accountant: 9709.0
Dabbler_Nervous: 7439.0
Dabbler_Collector: 8054.0
Coin_Flipper_Believer: 9357.0
Coin_Flipper_Once_in_a_While: 9746.0
Coin_Flipper_Two_Face: 9999.0
Coin_Flipper_Accountant: 9971.0
Coin_Flipper_Nervous: 9370.0
Coin_Flipper_Collector: 94

In [57]:
for col, val in results.loc["Mat_Player_ver_4"].items():
    print(f'{col}: {val}')

RandomPlayer: 9999.0
Saint_Sceptic_25_Naive: 10000.0
Saint_Sceptic_50_Naive: 10000.0
Saint_Sceptic_75_Naive: 10000.0
Saint_Sceptic_Adaptable_Naive: 8289.0
Mat_Player_ver_4: nan
YourName - not implemented Player: 10000.0
MyRandom_Default: 10000.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 9999.0
Random_Never_Accuse: 9920.0
Random_Always_Accuse: 10000.0
Mimic: 10000.0
Saint_Believer: 5970.0
Saint_Once_in_a_While: 7678.0
Saint_Two_Face: 9947.0
Saint_Accountant: 9397.0
Saint_Nervous: 5010.0
Mat_Player_ver_1: 4883.0
Mat_Player_ver_2: 5002.0
Mat_Player_ver_3: 7526.0
Adam_M_Player: 9984.0
AdamK_ver_1: 6566.0
AdamK2_ver_2: 6798.0
Saint_Collector: 6012.0
Dabbler_Believer: 8668.0
Dabbler_Once_in_a_While: 9491.0
Dabbler_Two_Face: 9997.0
Dabbler_Accountant: 9863.0
Dabbler_Nervous: 8403.0
Dabbler_Collector: 8751.0
Coin_Flipper_Believer: 9525.0
Coin_Flipper_Once_in_a_While: 9803.0
Coin_Flipper_Two_Face: 10000.0
Coin_Flipper_Accountant: 9977.0
Coin_Flipper_Nervous: 9496.0
Coin_Flipper_Collector